# Import packages

In [2]:
import numpy as numpy
import pandas as pd 
import json
from dateparser import parse

# Load file and preprocess

In [3]:
with open('../tda_positions.json', 'r') as f:
    positions = json.load(f)['securitiesAccount']['positions']
df_pos = pd.DataFrame(positions)
df_pos['cusip'] = df_pos.instrument.apply(lambda x: x['cusip'])

In [65]:
INSTRUCTION = {
    'BUY': 1,
    'SELL': -1,
}
df = pd.read_json('../tda_transactions.json')  
df = df[df.transactionSubType != 'IT']  # Ignore this, useless info
df['totalFees'] = df.fees.apply(lambda x: sum(x.values()))
df['cusip'] = df.transactionItem.apply(lambda x: x['instrument']['cusip'])
df['transactionDate'] = df.transactionDate.apply(lambda x: parse(x))

# For closed options
df_trade = df[df.type == 'TRADE']
# Extract orderId without extra data
df_trade['orderId_base'] = df_trade.orderId.apply(lambda x: str(x).split('.')[0])
# Create column for ticker symbol
df_trade['ticker'] = df_trade.transactionItem.apply(lambda x: x['instrument']['underlyingSymbol'])
df_trade['score'] = df_trade.transactionItem.apply(lambda x: x['amount'] * INSTRUCTION[x['instruction']])
df_trade['position_effect'] = df_trade.transactionItem.apply(lambda x: x['positionEffect'])
# df_trade = removePositions(df_trade, positions)

# For expired options
df_expired = df[df.type == 'RECEIVE_AND_DELIVER']
df_expired['score'] = df_expired.transactionItem.apply(lambda x: x['amount'])

df_trade.head()

,type,subAccount,settlementDate,orderId,netAmount,transactionDate,orderDate,transactionSubType,transactionId,cashBalanceEffectFlag,description,fees,transactionItem,totalFees,cusip,orderId_base,ticker,score,position_effect
0,TRADE,1,2021-02-16,2.114929e+09,-621.33,2021-02-12 20:55:58+00:00,2021-02-12T20:55:58+0000,BY,32599479264,True,BUY TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 2.0, 'price...",1.36,0APHA.CJ10020000,2114929104,APHA,2.0,OPENING
1,TRADE,1,2021-02-16,2.113482e+09,-187.66,2021-02-12 19:28:54+00:00,2021-02-12T19:28:54+0000,BY,32595505344,True,BUY TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 1.0, 'price...",0.67,0FUBO.BJ10050000,2113482419,FUBO,1.0,OPENING
2,TRADE,1,2021-02-16,2.112548e+09,589.33,2021-02-12 18:29:12+00:00,2021-02-12T18:29:12+0000,SL,32594042581,True,SELL TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 1.0, 'price...",0.69,0LAZR.CJ10040000,2112547960,LAZR,-1.0,CLOSING
3,TRADE,1,2021-02-12,2.105435e+09,-106.66,2021-02-11 20:50:03+00:00,2021-02-11T20:50:03+0000,BY,32550799584,True,BUY TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 1.0, 'price...",0.67,0DIS..BJ10220000,2105434757,DIS,1.0,OPENING
4,TRADE,1,2021-02-12,2.105036e+09,-205.66,2021-02-11 20:22:38+00:00,2021-02-11T20:22:38+0000,BY,32549921079,True,BUY TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 1.0, 'price...",0.67,0NPA..CJ10030000,2105035741,NPA,1.0,OPENING


# Group based on strategy

In [108]:
# Group by OrderId base, multi-legged orders will have a suffex with the same order id
# Ex: For spreads, 123456.1 and 123456.2 will be grouped together because there's 2
# orders with the same base order id 
df_grouped = df_trade.groupby(['orderId_base']).count()
counts = df_grouped.type.unique()
# Get index for different strategies
indices = {c: df_grouped[df_grouped.type == c].index.to_list() for c in counts}
# Get df for each strategy 
df_legs_raw = {c: df_trade[df_trade.orderId_base.isin(indices[c])] for c in counts}
# Regroup some dfs because some might not actually be 4 leg strategies
df_legs = {c: pd.DataFrame(columns=df_trade.columns) for c in counts}
for c in counts[counts > 2]:  # Skip basic and spread strategies because those are correct
    ticker_group = df_legs_raw[c].groupby(['ticker'])
    for ticker in ticker_group.groups.keys():
        group = ticker_group.get_group(ticker)
        # Use unique cusip to find out if there is actually the correct
        # amount of unique legs
        num_unique_cusip = len(group.groupby(['cusip']).groups.keys())
        # Add these to spread strategy 
        df_legs[num_unique_cusip] = df_legs[num_unique_cusip].append(group, ignore_index=True)

# Add original df_legs to fixed
for c in [1, 2]: 
    df_legs[c] = df_legs[c].append(df_legs_raw[c], ignore_index=True)

# Group Basic Options

In [172]:
df_legs[1].cusip

0      0FUBO.BJ10050000
1      0SPCE.BJ10055000
2      0TGT..BJ10200000
3      0TGT..BJ10200000
4      0SPCE.BJ10055000
             ...       
679    0AAL..J200013000
680    0INTC.II00053000
681    0AES..JG00019000
682    0SPY..I800342000
683    0INTC.II00053000
Name: cusip, Length: 684, dtype: object

In [31]:
group = df_legs[1].groupby(['cusip']).get_group('0NIO..BJ10065000').groupby(['position_effect'])
'BY' in group.get_group('OPENING').groupby(['transactionSubType']).groups

True

In [28]:
group.get_group('CLOSING').groupby(['transactionSubType']).get_group('SL')

,type,subAccount,settlementDate,orderId,netAmount,transactionDate,orderDate,transactionSubType,transactionId,cashBalanceEffectFlag,description,fees,transactionItem,totalFees,cusip,orderId_base,ticker,score,position_effect
64,TRADE,2,2021-02-02,2.021617e+09,432.66,2021-02-01 15:10:02+00:00,2021-02-01T15:09:59+0000,SL,31961629943,True,SELL TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 2.0, 'price...",1.38,0NIO..BJ10065000,2021616769,NIO,-2.0,CLOSING


In [117]:
def groupCompleteOrders(df, expired, cusip):
    """
    Combines opened and closed orders
    :return: dict for pairs and df for remaining
    """

    # Cases for incomplete orders
    if len(group) == 1 or 'CLOSING' not in group.position_effect.to_list():
        print('\t> Checking expired orders...')
        pairs, remaining = getPairs(df, expired, cusip)
        return pairs, remaining

    # Get open and closed orders
    position_effect = df.groupby(['position_effect'])
    opening = position_effect.get_group('OPENING').groupby(['transactionSubType'])
    closing = position_effect.get_group('CLOSING').groupby(['transactionSubType'])

    all_pairs = []
    remaining1 = pd.DataFrame(columns=group.columns)
    remaining2 = pd.DataFrame(columns=group.columns)

    if 'BY' in opening.groups: 
        BTO = opening.get_group('BY')
        STC = closing.get_group('SL')
        data = BTO.append(STC).sort_values(by=['transactionDate'], ascending=True)
        pairs, remaining1 = getPairs(data, expired, cusip)
        all_pairs += pairs

    if 'SL' in opening.groups: 
        STO = opening.get_group('SL')
        BTC = closing.get_group('BY')
        data = STO.append(BTC).sort_values(by=['transactionDate'], ascending=True)
        pairs, remaining2 = getPairs(data, expired, cusip)
        all_pairs += pairs
    
    remaining = remaining1.append(remaining2)
    return all_pairs, remaining

def getPairs(df, expired, cusip):
    counter = 0
    pair = []
    pairs = []
    for idx, row in df.iterrows():
        counter += row.score
        pair.append(idx)

        # Save pair and reinit
        if counter == 0:
            pairs.append(pair)
            pair = []

    # Extract actual rows
    pairs = [df.loc[p] for p in pairs]
    remainder = pd.DataFrame(columns=df.columns)

    if counter != 0:
        # Extract remaining orders
        remainder = df.loc[pair]
        group = expired[expired.cusip == cusip]
        # Just check if the leftover score is in df_expired
        if abs(counter) == group.score.sum():
            pairs.append(remainder.append(group))
            # Remove resolved expired orders
            print('\t> Resolved expired!')
            expired.drop(group.index)
            remainder = pd.DataFrame(columns=df.columns)
        else:
            print('\t> Added to remainder')
            print(remainder)
            print()

    return pairs, remainder


groups = df_legs[1].groupby(['cusip'])
basicOrders = []
remainders = pd.DataFrame(columns=df_trade.columns)
# Loop through each cusip group
for cusip in groups.groups.keys():
    print(cusip)
    # Sort by date, oldest -> recent 
    group = groups.get_group(cusip)
    # Find complete orders
    pairs, remainder = groupCompleteOrders(group, df_expired, cusip) 
    basicOrders += pairs
    remainders = remainders.append(remainder)

0AAL..J200013000
0AAPL.BC10137000
0AAPL.C510140000
	> Checking expired orders
3.0
REMAINDER FOUND:
     type subAccount settlementDate       orderId  netAmount  \
48  TRADE          1     2021-02-09  2.068868e+09    -913.99   

              transactionDate                 orderDate transactionSubType  \
48  2021-02-08 16:08:50+00:00  2021-02-08T16:08:49+0000                 BY   

   transactionId cashBalanceEffectFlag description  \
48   32186081337                  True   BUY TRADE   

                                                 fees  \
48  {'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...   

                                      transactionItem  totalFees  \
48  {'accountId': 275422611, 'amount': 3.0, 'price...       2.03   

               cusip orderId_base ticker  score position_effect  
48  0AAPL.C510140000   2068867797   AAPL    3.0         OPENING  

0AAPL.JN00130000
0AAPL.LV00130000
0AAPL.WD00116000
0ACB..AM10010500
0ACB..LB00012000
0ACB..LV00011000
0AES..JG00019000
0A

In [75]:
group = df_legs[1].groupby(['cusip']).get_group('0AMC..B510040000')
position_effect = group.groupby(['position_effect'])
opening = position_effect.get_group('OPENING').groupby(['transactionSubType'])
closing = position_effect.get_group('CLOSING').groupby(['transactionSubType'])
all_pairs = []
remaining1 = pd.DataFrame(columns=group.columns)
remaining2 = pd.DataFrame(columns=group.columns)
if 'BY' in opening.groups: 
    BTO = opening.get_group('BY')
    STC = closing.get_group('SL')
    data = BTO.append(STC).sort_values(by=['transactionDate'], ascending=True)
    pairs, remaining1 = getPairs(data, df_expired, '0NIO..BJ10065000')
    all_pairs += pairs
if 'SL' in opening.groups: 
    STO = opening.get_group('SL')
    BTC = closing.get_group('BY')
    data = STO.append(BTC).sort_values(by=['transactionDate'], ascending=True)
    pairs, remaining2 = getPairs(data, expired, cusip)
    all_pairs += pairs
remainder = remaining1.append(remaining2)

KeyError: 'CLOSING'

# Redo but grouping by cusip first

In [56]:
def removePositions(df, positions, cusip):
    """ Remove open positions """
    if cusip in df_pos.cusip:
        order_ids = [df[df.cusip == p['instrument']['cusip']].sort_values(by=['transactionSubType'], ascending=False).iloc[0].orderId for p in positions]
        idx_remove = df[df.orderId.isin(order_ids)].index
        return df.drop(idx_remove)
    return df

INSTRUCTION = {
    'BUY': 1,
    'SELL': -1
}
POSITION_EFFECT = {
    'OPEN': 1,
    'CLOSE': -1
}

In [142]:
df = pd.read_json('../tda_transactions.json')  # Assumes file has no open positions
df['totalFees'] = df.fees.apply(lambda x: sum(x.values()))
df['cusip'] = df.transactionItem.apply(lambda x: x['instrument']['cusip'])
df['transactionDate'] = df.transactionDate.apply(lambda x: parse(x))

gdf = df.groupby(['cusip'])
cusips = list(gdf.groups.keys())
gdf.groups

{'0AAL..J200013000': [1109, 1135], '0AAPL.AT10130000': [245, 248, 334, 337], '0AAPL.AT10131000': [246, 249, 335, 336], '0AAPL.AT10145000': [124, 127, 164], '0AAPL.AT10150000': [126, 129, 166], '0AAPL.AT10155000': [125, 128, 165], '0AAPL.BC10137000': [53, 55], '0AAPL.C510140000': [5, 57], '0AAPL.IB00120000': [1251, 1294], '0AAPL.IB00121250': [1252, 1293], '0AAPL.JN00117000': [979, 1028], '0AAPL.JN00118000': [980, 1027], '0AAPL.JN00130000': [961, 969], '0AAPL.JU00111000': [1032, 1160], '0AAPL.JU00112000': [1031, 1161], '0AAPL.LV00130000': [498, 499, 573], '0AAPL.WD00116000': [756, 759], '0ABNB.AM10155000': [329, 339], '0ABNB.AM10157500': [330, 338], '0ACB..AM10010500': [341, 373], '0ACB..LB00012000': [609, 612], '0ACB..LV00011000': [472, 491], '0ACN..JG00215000': [1133, 1142], '0ACN..JG00217500': [1134, 1143], '0ADBE.AM10480000': [277, 324], '0ADBE.AM10482500': [278, 325], '0ADBE.MF10465000': [284, 406], '0ADBE.MF10470000': [283, 405], '0AES..JG00019000': [1010, 1255], '0AES..LI00022000'

In [175]:
def splitExpiredOrders(df, cusip):
    # For manually closed orders
    df_trade = df[df.type == 'TRADE']
    # Extract orderId without extra data
    df_trade['orderId_base'] = df_trade.orderId.apply(lambda x: str(x).split('.')[0])
    # Create column for ticker symbol
    df_trade['ticker'] = df_trade.transactionItem.apply(lambda x: x['instrument']['underlyingSymbol'])
    df_trade['score'] = df_trade.transactionItem.apply(lambda x: x['amount'] * INSTRUCTION[x['instruction']])
    df_trade['position_effect'] = df_trade.transactionItem.apply(lambda x: x['positionEffect'])
    df_trade = removePositions(df_trade, positions, cusip)

    # For expired options
    df_expired = df[df.type == 'RECEIVE_AND_DELIVER']
    df_expired['score'] = df_expired.transactionItem.apply(lambda x: x['amount'])

    return df_trade, df_expired

def splitByStrategy(df_trade):
    # Group by OrderId base, multi-legged orders will have a suffex with the same order id
    # Ex: For spreads, 123456.1 and 123456.2 will be grouped together because there's 2
    # orders with the same base order id 
    df_grouped = df_trade.groupby(['orderId_base']).count()
    counts = df_grouped.type.unique()
    # Get index for different strategies
    indices = {c: df_grouped[df_grouped.type == c].index.to_list() for c in counts}
    # Get df for each strategy 
    df_legs_raw = {c: df_trade[df_trade.orderId_base.isin(indices[c])] for c in counts}
    # Regroup some dfs because some might not actually be 4 leg strategies
    df_legs = {c: pd.DataFrame(columns=df_trade.columns) for c in counts}
    for c in counts[counts > 2]:  # Skip basic and spread strategies because those are correct
        ticker_group = df_legs_raw[c].groupby(['ticker'])
        for ticker in ticker_group.groups.keys():
            group = ticker_group.get_group(ticker)
            # Use unique cusip to find out if there is actually the correct 
            # amount of unique legs
            num_unique_cusip = len(group.groupby(['cusip']).groups.keys())
            # Add these to spread strategy 
            df_legs[num_unique_cusip] = df_legs[num_unique_cusip].append(group, ignore_index=True)

    # Add original df_legs to fixed
    for c in counts[counts <= 2]: 
        df_legs[c] = df_legs[c].append(df_legs_raw[c], ignore_index=True)
    
    return df_legs

cusip = '0DKNG.J900055000'
g = gdf.get_group(cusip)
trd, exp = splitExpiredOrders(g, cusip)
legs = splitByStrategy(trd)

In [179]:
legs[1]

,type,subAccount,settlementDate,orderId,netAmount,transactionDate,orderDate,transactionSubType,transactionId,cashBalanceEffectFlag,description,fees,transactionItem,totalFees,cusip,orderId_base,ticker,score,position_effect
0,TRADE,2,2020-10-02,1.535939e+09,-720.66,2020-10-01 16:11:36+00:00,2020-10-01T16:06:44+0000,BY,29194378390,True,BUY TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 1.0, 'price...",0.67,0DKNG.J900055000,1535938950,DKNG,1.0,CLOSING
1,TRADE,2,2020-09-29,1.523605e+09,-498.66,2020-09-28 16:39:23+00:00,2020-09-28T16:39:23+0000,BY,29071686310,True,BUY TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 1.0, 'price...",0.67,0DKNG.J900055000,1523605449,DKNG,1.0,CLOSING


In [174]:
df_legs[2]

,type,subAccount,settlementDate,orderId,netAmount,transactionDate,orderDate,transactionSubType,transactionId,cashBalanceEffectFlag,description,fees,transactionItem,totalFees,cusip,orderId_base,ticker,score,position_effect
0,TRADE,2,2020-09-23,1.504497e+09,-431.32,2020-09-22 15:33:00+00:00,2020-09-22T15:33:00+0000,BY,28954170746,True,BUY TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 1.0, 'price...",1.34,0DKNG.J900054000,1504496655,DKNG,1.0,OPENING
1,TRADE,2,2020-09-23,1.504497e+09,-428.01,2020-09-22 15:33:00+00:00,2020-09-22T15:33:00+0000,BY,28954170787,True,BUY TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 1.0, 'price...",0.02,0DKNG.J900054000,1504496655,DKNG,1.0,OPENING
2,TRADE,2,2020-09-23,1.504497e+09,380.67,2020-09-22 15:33:00+00:00,2020-09-22T15:33:00+0000,SL,28954170965,True,SELL TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 1.0, 'price...",1.36,0DKNG.J900055000,1504496655,DKNG,-1.0,OPENING
3,TRADE,2,2020-09-23,1.504497e+09,380.98,2020-09-22 15:33:00+00:00,2020-09-22T15:33:00+0000,SL,28954170999,True,SELL TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 1.0, 'price...",0.04,0DKNG.J900055000,1504496655,DKNG,-1.0,OPENING
4,TRADE,2,2021-01-11,1.893203e+09,-623.31,2021-01-08 15:41:16+00:00,2021-01-08T15:41:15+0000,BY,31327548948,True,BUY TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 1.0, 'price...",1.32,0ETSY.AM10180000,1893203327,ETSY,1.0,OPENING
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,TRADE,2,2020-09-04,1.447613e+09,-47.33,2020-09-03 19:36:02+00:00,2020-09-03T19:31:32+0000,BY,28613729316,True,BUY TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 2.0, 'price...",1.36,0SPY..I400353000,1447612872,SPY,2.0,OPENING
524,TRADE,2,2020-09-04,1.446187e+09,170.33,2020-09-03 16:53:18+00:00,2020-09-03T16:53:16+0000,SL,28610609151,True,SELL TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 1.0, 'price...",0.69,0SPY..I400348000,1446186688,SPY,-1.0,OPENING
525,TRADE,2,2020-09-04,1.446187e+09,-61.67,2020-09-03 16:53:18+00:00,2020-09-03T16:53:16+0000,BY,28610610639,True,BUY TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 1.0, 'price...",0.69,0SPY..I400352000,1446186688,SPY,1.0,OPENING
526,TRADE,2,2020-09-04,1.446186e+09,359.88,2020-09-03 16:42:11+00:00,2020-09-03T16:42:11+0000,SL,28610333226,True,SELL TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 1.0, 'price...",1.14,0VIX..IG00032500,1446186441,VIX,-1.0,OPENING


----
# Just group by CUSIP and find completed orders

In [66]:
POSITION = {
    'OPENING': 1,
    'CLOSING': -1
}
df['position_effect'] = df.transactionItem.apply(lambda x: x['amount'] * POSITION[x.get('positionEffect', 'CLOSING')])
df_groups = df.groupby(['cusip'])

In [74]:
cusips = list(df_groups.groups.keys())
df_open = pd.DataFrame(columns=df.columns)
df_closed = pd.DataFrame(columns=df.columns)
# Loop through each cusip group
for cusip in cusips:
    # print(cusip)
    group = df_groups.get_group(cusip).sort_values(by=['transactionDate'])
    group['cusip'] = cusip
    
    counter = 0
    open_positions = []
    for idx, row in group.iterrows():
        counter += row.position_effect
        open_positions.append(idx)

        # Save pair and reinit
        if counter == 0:
            open_positions = []

    # Add to df
    open = group[group.index.isin(open_positions)]
    df_open = df_open.append(open, ignore_index=True)
    closed = group[~group.index.isin(open_positions)]
    df_closed = df_closed.append(open, ignore_index=True)

In [76]:
df_open

,type,subAccount,settlementDate,orderId,netAmount,transactionDate,orderDate,transactionSubType,transactionId,cashBalanceEffectFlag,description,fees,transactionItem,totalFees,cusip,position_effect
0,TRADE,1,2021-02-09,2.068868e+09,-913.99,2021-02-08 16:08:50+00:00,2021-02-08T16:08:49+0000,BY,32186081337,True,BUY TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 3.0, 'price...",2.03,0AAPL.C510140000,3.0
1,TRADE,1,2021-02-12,2.105036e+09,-187.66,2021-02-11 20:21:48+00:00,2021-02-11T20:21:35+0000,BY,32549867239,True,BUY TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 1.0, 'price...",0.67,0AAPL.C510140000,1.0
2,TRADE,1,2021-02-16,2.114929e+09,-621.33,2021-02-12 20:55:58+00:00,2021-02-12T20:55:58+0000,BY,32599479264,True,BUY TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 2.0, 'price...",1.36,0APHA.CJ10020000,2.0
3,TRADE,1,2021-02-09,2.073412e+09,-860.66,2021-02-08 20:47:30+00:00,2021-02-08T20:47:24+0000,BY,32194643318,True,BUY TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 1.0, 'price...",0.67,0CCIV.CJ10040000,1.0
4,TRADE,1,2021-02-12,2.105435e+09,-106.66,2021-02-11 20:50:03+00:00,2021-02-11T20:50:03+0000,BY,32550799584,True,BUY TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 1.0, 'price...",0.67,0DIS..BJ10220000,1.0
5,TRADE,1,2021-02-16,2.113482e+09,-187.66,2021-02-12 19:28:54+00:00,2021-02-12T19:28:54+0000,BY,32595505344,True,BUY TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 1.0, 'price...",0.67,0FUBO.BJ10050000,1.0
6,TRADE,1,2021-02-10,2.081752e+09,-625.66,2021-02-09 18:50:26+00:00,2021-02-09T18:50:25+0000,BY,32236445199,True,BUY TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 1.0, 'price...",0.67,0JKS..CJ10070000,1.0
7,TRADE,1,2021-02-10,2.081316e+09,-495.66,2021-02-09 18:22:58+00:00,2021-02-09T18:22:41+0000,BY,32235541474,True,BUY TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 1.0, 'price...",0.67,0NPA..CJ10030000,1.0
8,TRADE,1,2021-02-10,2.082214e+09,-320.66,2021-02-09 19:22:08+00:00,2021-02-09T19:22:08+0000,BY,32236955573,True,BUY TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 1.0, 'price...",0.67,0NPA..CJ10030000,1.0
9,TRADE,1,2021-02-12,2.105036e+09,-205.66,2021-02-11 20:22:38+00:00,2021-02-11T20:22:38+0000,BY,32549921079,True,BUY TRADE,"{'rFee': 0.0, 'additionalFee': 0.0, 'cdscFee':...","{'accountId': 275422611, 'amount': 1.0, 'price...",0.67,0NPA..CJ10030000,1.0


In [82]:
df.transactionItem.apply(lambda x: parse(x['instrument']['optionExpirationDate']))

0       2021-03-19 05:00:00+00:00
1       2021-02-19 06:00:00+00:00
2       2021-03-19 05:00:00+00:00
3       2021-02-19 06:00:00+00:00
4       2021-03-19 05:00:00+00:00
                  ...            
1304    2020-09-04 05:00:00+00:00
1305    2020-09-04 05:00:00+00:00
1306    2020-09-16 05:00:00+00:00
1307    2020-09-16 05:00:00+00:00
1308    2020-09-18 05:00:00+00:00
Name: transactionItem, Length: 1305, dtype: object